In [218]:
#Importing Packages

import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import nfl_data_py as nfl
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [219]:
#Specifying Date Ranges

current_year = datetime.date.today().year
years = range(2015,current_year)

In [220]:
#Importing API
seasonal_data_df = nfl.import_seasonal_data(years)

In [221]:
seasonal_data_df

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,00-0007091,2015,REG,156,256,1690.0,9,5.0,16.0,101.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.159657
1,00-0010346,2015,REG,198,331,2249.0,9,17.0,16.0,95.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.113044
2,00-0018227,2015,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003743
3,00-0019596,2015,REG,402,624,4770.0,36,7.0,38.0,225.0,...,0.011866,0.003519,0.007481,0.0,0.004367,0.003774,0.003741,0.005985,0.057234,0.210173
4,00-0019596,2016,REG,291,432,3554.0,28,2.0,15.0,87.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.201465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,00-0039150,2023,REG,315,527,2877.0,11,10.0,62.0,477.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.154486
5487,00-0039152,2023,REG,149,255,1808.0,8,4.0,28.0,185.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.159821
5488,00-0039163,2023,REG,319,499,4108.0,23,5.0,38.0,331.0,...,0.000562,0.002783,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.206779
5489,00-0039164,2023,REG,50,84,577.0,3,1.0,7.0,29.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.213689


In [222]:
#Importing CSV from Rookie Perdictions. A lot of good data we can use for this prediction
college_profile_df = pd.read_csv("https://raw.githubusercontent.com/JoshKepler/Portfolio/main/NFL%20Project/Data/unfiltered_rookie_proj.csv")

C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\3531356292.py:2: DtypeWarning: Columns (181,183,184,372,374,375,376,394,395,397,453,455,456,457) have mixed types. Specify dtype option on import or set low_memory=False.
  college_profile_df = pd.read_csv("https://raw.githubusercontent.com/JoshKepler/Portfolio/main/NFL%20Project/Data/unfiltered_rookie_proj.csv")


In [223]:
college_profile_df

,Unnamed: 0,index,season,cfb_id,player_name,pos,school,ht,wt,forty,...,hits_y_pred_gbf,defensive_sacks_y_pred_gbf,safeties_y_pred_gbf,defensive_interceptions_y_pred_gbf,defensive_interception_touchdowns_y_pred_gbf,pass_break_ups_y_pred_gbf,coverage_targets_y_pred_gbf,allowed_recpetions_y_pred_gbf,allowed_passing_yards_y_pred_gbf,allowed_passing_touchdowns_y_pred_gbf
0,0,0.0,2015.0,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.600,...,1.316581e-14,2.632835e-15,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000
1,2,2.0,2015.0,jay-ajayi-1,Jay Ajayi,RB,Boise State,72.0,221.0,4.570,...,1.316581e-14,2.632835e-15,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000
2,5,5.0,2015.0,javorius-allen-1,Javorius Allen,RB,USC,72.0,221.0,4.530,...,1.316126e-14,2.632835e-15,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000
3,12,12.0,2015.0,cameron-artis-payne-1,Cameron Artis-Payne,RB,Auburn,70.0,212.0,4.530,...,1.316581e-14,2.632835e-15,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000
4,22,62.0,2015.0,0,John Crockett,RB,North Dakota State,72.0,217.0,4.620,...,1.316126e-14,2.632835e-15,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,3108,3093.0,2024.0,drake-nugent-1,Drake Nugent,C,Michigan,74.0,298.0,5.230,...,0.000000e+00,0.000000e+00,0.0,0.000011,0.0,0.0001,0.000661,0.000339,0.004061,0.000011
3200,3118,3105.0,2024.0,jackson-powers-johnson-1,Jackson Powers-Johnson,C,Oregon,75.0,328.0,5.225,...,0.000000e+00,0.000000e+00,0.0,0.000011,0.0,0.0001,0.000661,0.000339,0.004061,0.000011
3201,3126,3114.0,2024.0,andrew-raym-1,Andrew Raym,C,Oklahoma,76.0,314.0,5.420,...,0.000000e+00,0.000000e+00,0.0,0.000011,0.0,0.0001,0.000661,0.000339,0.004061,0.000011
3202,3142,3132.0,2024.0,nick-samac-1,Nick Samac,C,Michigan St.,76.0,307.0,5.225,...,0.000000e+00,0.000000e+00,0.0,0.000011,0.0,0.0001,0.000661,0.000339,0.004061,0.000011


In [224]:
#Importing from API, ID key
id_df = nfl.import_ids()

In [225]:
#Creating columns to 
id_df = id_df[['gsis_id', 'pff_id','cfbref_id', 'name', 'merge_name', 'position', 'team', 'birthdate', 'age',
       'draft_year', 'draft_round', 'draft_pick', 'draft_ovr', 'height', 'weight', 'college']]

id_df = id_df[id_df['draft_year'] > 2014]

id_df['2nd_season'] = (id_df['draft_year'] + 1).astype('int')
id_df['3rd_season'] = (id_df['draft_year'] + 2).astype('int')

id_df

,gsis_id,pff_id,cfbref_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,height,weight,college,2nd_season,3rd_season
0,NaN,NaN,NaN,Caleb Williams,caleb williams,QB,CHI,2001-11-18,22.6,2024.0,1.0,1.0,NaN,73.0,215.0,USC,2025,2026
1,NaN,NaN,NaN,Drake Maye,drake maye,QB,NEP,2002-08-30,21.9,2024.0,1.0,3.0,NaN,76.0,225.0,North Carolina,2025,2026
2,NaN,NaN,NaN,Jayden Daniels,jayden daniels,QB,WAS,2000-12-18,23.6,2024.0,1.0,2.0,NaN,76.0,210.0,LSU,2025,2026
3,NaN,NaN,NaN,Bo Nix,bo nix,QB,DEN,2000-02-25,24.4,2024.0,1.0,12.0,NaN,74.0,217.0,Oregon,2025,2026
4,NaN,NaN,NaN,Michael Penix Jr.,michael penix,QB,ATL,2000-05-08,24.2,2024.0,1.0,8.0,NaN,74.0,216.0,Washington,2025,2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908,00-0032298,NaN,NaN,Bronson Hill,bronson hill,RB,FA,1993-01-02,28.3,2015.0,NaN,NaN,NaN,71.0,221.0,Eastern Michigan,2016,2017
3909,00-0031866,9962.0,NaN,Jordan Williams,jordan williams,DE,NYG,1993-03-23,28.1,2015.0,NaN,NaN,NaN,76.0,262.0,Tennessee,2016,2017
3910,00-0033138,NaN,NaN,Sam Ficken,sam ficken,PK,FA,1992-12-14,30.2,2015.0,NaN,NaN,NaN,73.0,192.0,Penn State,2016,2017
3911,00-0031898,9994.0,NaN,Alex Singleton,alex singleton,LB,DEN,1993-12-07,30.6,2015.0,NaN,NaN,NaN,74.0,240.0,Montana State,2016,2017


In [226]:
college_profile_df = college_profile_df.merge(id_df[['gsis_id', '2nd_season', '3rd_season']], left_on='player_id', right_on='gsis_id', suffixes=['','_y'])

In [227]:
college_profile_df = college_profile_df.loc[:, ~college_profile_df.columns.str.contains("_y_pred_gbf")]
college_profile_df

,Unnamed: 0,index,season,cfb_id,player_name,pos,school,ht,wt,forty,...,defensive_interceptions,defensive_interception_touchdowns,pass_break_ups,coverage_targets,allowed_recpetions,allowed_passing_yards,allowed_passing_touchdowns,gsis_id_y,2nd_season,3rd_season
0,0,0.0,2015.0,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0032104,2016,2017
1,2,2.0,2015.0,jay-ajayi-1,Jay Ajayi,RB,Boise State,72.0,221.0,4.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0031590,2016,2017
2,5,5.0,2015.0,javorius-allen-1,Javorius Allen,RB,USC,72.0,221.0,4.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0031577,2016,2017
3,12,12.0,2015.0,cameron-artis-payne-1,Cameron Artis-Payne,RB,Auburn,70.0,212.0,4.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0031599,2016,2017
4,307,48.0,2015.0,david-cobb-1,David Cobb,RB,Minnesota,71.0,229.0,4.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0032133,2016,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,1558,1441.0,2019.0,david-long-3,David Long,CB,Michigan,71.0,196.0,4.45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00-0035665,2020,2021
573,2069,1967.0,2021.0,brandin-echols-1,Brandin Echols,CB,Kentucky,70.0,179.0,4.34,...,2.0,1.0,5.0,73.0,47.0,600.0,1.0,00-0036504,2022,2023
574,2339,2239.0,2021.0,avery-williams-6,Avery Williams,CB,Boise St.,68.0,187.0,4.43,...,0.0,0.0,0.0,15.0,12.0,109.0,1.0,00-0036950,2022,2023
575,2352,2253.0,2021.0,nahshon-wright-1,Nahshon Wright,CB,Oregon St.,76.0,183.0,4.46,...,0.0,0.0,0.0,8.0,5.0,81.0,0.0,00-0036986,2022,2023


In [228]:
college_profile_df['season'] = college_profile_df['season'].astype('int')

college_profile_df['player_id_1st_season'] = college_profile_df['player_id'] + "_" + college_profile_df['season'].astype('str')
college_profile_df['player_id_2nd_season'] = college_profile_df['player_id'] + "_" + college_profile_df['2nd_season'].astype('str')
college_profile_df['player_id_3rd_season'] = college_profile_df['player_id'] + "_" + college_profile_df['3rd_season'].astype('str')
college_profile_df = college_profile_df.sort_values(by='season')

college_profile_df

,Unnamed: 0,index,season,cfb_id,player_name,pos,school,ht,wt,forty,...,coverage_targets,allowed_recpetions,allowed_passing_yards,allowed_passing_touchdowns,gsis_id_y,2nd_season,3rd_season,player_id_1st_season,player_id_2nd_season,player_id_3rd_season
0,0,0.0,2015,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.600,...,0.0,0.0,0.0,0.0,00-0032104,2016,2017,00-0032104_2015,00-0032104_2016,00-0032104_2017
152,354,105.0,2015,devin-funchess-1,Devin Funchess,WR,Michigan,76.0,232.0,4.700,...,0.0,0.0,0.0,0.0,00-0032055,2016,2017,00-0032055_2015,00-0032055_2016,00-0032055_2017
151,335,83.0,2015,phillip-dorsett-1,Phillip Dorsett,WR,Miami (FL),70.0,185.0,4.330,...,0.0,0.0,0.0,0.0,00-0032208,2016,2017,00-0032208_2015,00-0032208_2016,00-0032208_2017
150,332,80.0,2015,stefon-diggs-1,Stefon Diggs,WR,Maryland,72.0,195.0,4.460,...,0.0,0.0,0.0,0.0,00-0031588,2016,2017,00-0031588_2015,00-0031588_2016,00-0031588_2017
149,324,70.0,2015,geremy-davis-1,Geremy Davis,WR,Connecticut,74.0,216.0,4.490,...,0.0,0.0,0.0,0.0,00-0031953,2016,2017,00-0031953_2015,00-0031953_2016,00-0031953_2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2810,2762.0,2023,jonathan-mingo-1,Jonathan Mingo,WR,Mississippi,74.0,220.0,4.460,...,0.0,0.0,0.0,0.0,00-0039062,2024,2025,00-0039062_2023,00-0039062_2024,00-0039062_2025
361,2809,2761.0,2023,marvin-mims-1,Marvin Mims,WR,Oklahoma,71.0,183.0,4.380,...,0.0,0.0,0.0,0.0,00-0038976,2024,2025,00-0038976_2023,00-0038976_2024,00-0038976_2025
359,2765,2708.0,2023,quentin-johnston-1,Quentin Johnston,WR,TCU,75.0,208.0,4.505,...,0.0,0.0,0.0,0.0,00-0038544,2024,2025,00-0038544_2023,00-0038544_2024,00-0038544_2025
373,2905,2864.0,2023,dontayvion-wicks-1,Dontayvion Wicks,WR,Virginia,73.0,206.0,4.620,...,0.0,0.0,0.0,0.0,00-0038393,2024,2025,00-0038393_2023,00-0038393_2024,00-0038393_2025


In [229]:
seasonal_data_df['player_id_season'] = seasonal_data_df['player_id'] + "_" + seasonal_data_df['season'].astype('str')
seasonal_data_df

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,player_id_season
0,00-0007091,2015,REG,156,256,1690.0,9,5.0,16.0,101.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.159657,00-0007091_2015
1,00-0010346,2015,REG,198,331,2249.0,9,17.0,16.0,95.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.113044,00-0010346_2015
2,00-0018227,2015,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003743,00-0018227_2015
3,00-0019596,2015,REG,402,624,4770.0,36,7.0,38.0,225.0,...,0.003519,0.007481,0.0,0.004367,0.003774,0.003741,0.005985,0.057234,0.210173,00-0019596_2015
4,00-0019596,2016,REG,291,432,3554.0,28,2.0,15.0,87.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.201465,00-0019596_2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,00-0039150,2023,REG,315,527,2877.0,11,10.0,62.0,477.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.154486,00-0039150_2023
5487,00-0039152,2023,REG,149,255,1808.0,8,4.0,28.0,185.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.159821,00-0039152_2023
5488,00-0039163,2023,REG,319,499,4108.0,23,5.0,38.0,331.0,...,0.002783,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.206779,00-0039163_2023
5489,00-0039164,2023,REG,50,84,577.0,3,1.0,7.0,29.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.213689,00-0039164_2023


# Creating 2nd Year Projections

In [230]:
second_year_df = college_profile_df.merge(seasonal_data_df, left_on='player_id_2nd_season', right_on='player_id_season', suffixes=['_1st_year_stats', '_2nd_year_stats'], how='outer')
second_year_df = second_year_df.fillna(0)
second_year_df = second_year_df[second_year_df['player_name'] != 0]
second_year_df

,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,wopr_y_2nd_year_stats,ry_sh_2nd_year_stats,rtd_sh_2nd_year_stats,rfd_sh_2nd_year_stats,rtdfd_sh_2nd_year_stats,dom_2nd_year_stats,w8dom_2nd_year_stats,yptmpa_2nd_year_stats,ppr_sh_2nd_year_stats,player_id_season
0,0.0,0.0,2015.0,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.600,...,0.099125,0.095000,0.250000,0.129032,0.142857,0.172500,0.126000,0.721519,0.126236,00-0032104_2016
1,354.0,105.0,2015.0,devin-funchess-1,Devin Funchess,WR,Michigan,76.0,232.0,4.700,...,0.293134,0.107630,0.200000,0.119048,0.127660,0.153815,0.126104,0.737575,0.074637,00-0032055_2016
2,335.0,83.0,2015.0,phillip-dorsett-1,Phillip Dorsett,WR,Miami (FL),70.0,185.0,4.330,...,0.299182,0.127598,0.068966,0.098039,0.094421,0.098282,0.115871,0.970588,0.069724,00-0032208_2016
3,332.0,80.0,2015.0,stefon-diggs-1,Stefon Diggs,WR,Maryland,72.0,195.0,4.460,...,0.571465,0.267635,0.200000,0.271676,0.265957,0.233817,0.254108,1.854209,0.179204,00-0031588_2016
4,324.0,70.0,2015.0,geremy-davis-1,Geremy Davis,WR,Connecticut,74.0,216.0,4.490,...,0.040645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,00-0031953_2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,2810.0,2762.0,2023.0,jonathan-mingo-1,Jonathan Mingo,WR,Mississippi,74.0,220.0,4.460,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
573,2809.0,2761.0,2023.0,marvin-mims-1,Marvin Mims,WR,Oklahoma,71.0,183.0,4.380,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
574,2765.0,2708.0,2023.0,quentin-johnston-1,Quentin Johnston,WR,TCU,75.0,208.0,4.505,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
575,2905.0,2864.0,2023.0,dontayvion-wicks-1,Dontayvion Wicks,WR,Virginia,73.0,206.0,4.620,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [231]:
stats_to_perdict = [ 'games_2nd_year_stats',
 'passing_yards_2nd_year_stats',
 'passing_tds_2nd_year_stats',
 'interceptions_2nd_year_stats',
 'sacks_2nd_year_stats',
 'sack_yards_2nd_year_stats',
 'passing_air_yards_2nd_year_stats',
 'passing_yards_after_catch_2nd_year_stats',
 'passing_first_downs_2nd_year_stats',
 'passing_epa_2nd_year_stats',
 'pacr_2nd_year_stats',
 'dakota_2nd_year_stats',
 'rushing_yards_2nd_year_stats',
 'rushing_tds_2nd_year_stats',
 'rushing_fumbles_2nd_year_stats',
 'rushing_fumbles_lost_2nd_year_stats',
 'rushing_first_downs_2nd_year_stats',
 'rushing_epa_2nd_year_stats',
 'targets_2nd_year_stats',
 'receptions_2nd_year_stats',
 'receiving_yards_2nd_year_stats',
 'receiving_tds_2nd_year_stats',
 'receiving_fumbles_2nd_year_stats',
 'receiving_fumbles_lost_2nd_year_stats',
 'receiving_air_yards_2nd_year_stats',
 'receiving_yards_after_catch_2nd_year_stats',
 'receiving_first_downs_2nd_year_stats',
 'receiving_epa_2nd_year_stats',
 'racr_2nd_year_stats',
 'air_yards_share_2nd_year_stats',
 'wopr_x_2nd_year_stats',
 'special_teams_tds_2nd_year_stats',
 'tgt_sh_2nd_year_stats',
 'ay_sh_2nd_year_stats',
 'yac_sh_2nd_year_stats',
 'wopr_y_2nd_year_stats',
 'ry_sh_2nd_year_stats',
 'rtd_sh_2nd_year_stats',
 'rfd_sh_2nd_year_stats',
 'dom_2nd_year_stats',
 'yptmpa_2nd_year_stats']

x_stats = ['ht',
 'wt',
 'forty',
 'bench',
 'vertical',
 'broad_jump',
 'cone',
 'shuttle',
 'draft_ovr',
 'draft_round',
 'Season_career',
 'player_game_count_career',
 'aimed_passes_career',
 'attempts_passing_career',
 'avg_depth_of_target_passing_career',
 'avg_time_to_throw_career',
 'bats_career',
 'big_time_throws_career',
 'completions_career',
 'declined_penalties_career',
 'def_gen_pressures_career',
 'dropbacks_career',
 'drops_passing_career',
 'first_downs_passing_career',
 'grades_hands_fumble_career',
 'grades_offense_career',
 'grades_pass_career',
 'hit_as_threw_career',
 'interceptions_passing_career',
 'passing_snaps_career',
 'penalties_career',
 'pressure_to_sack_rate_career',
 'qb_rating_career',
 'sacks_career',
 'scrambles_career',
 'spikes_career',
 'thrown_aways_career',
 'touchdowns_passing_career',
 'turnover_worthy_plays_career',
 'yards_passing_career',
 'attempts_rushing_career',
 'avoided_tackles_career',
 'breakaway_attempts_career',
 'breakaway_yards_career',
 'designed_yards_career',
 'elu_recv_mtf_career',
 'elu_rush_mtf_career',
 'elu_yco_career',
 'elusive_rating_career',
 'explosive_career',
 'first_downs_rushing_career',
 'fumbles_career',
 'gap_attempts_career',
 'grades_run_career',
 'grades_offense_penalty_career',
 'grades_run_block_career',
 'longest_rush_career',
 'run_plays_career',
 'scramble_yards_career',
 'total_touches_career',
 'touchdowns_rushing_career',
 'yards_rushing_career',
 'yards_after_contact_career',
 'yco_attempt_career',
 'ypa_career',
 'zone_attempts_career',
 'avg_depth_of_target_receiving_career',
 'contested_receptions_career',
 'contested_targets_career',
 'drops_receiving_career',
 'first_downs_receiving_career',
 'grades_hands_drop_career',
 'grades_pass_route_career',
 'inline_rate_career',
 'inline_snaps_career',
 'interceptions_receiving_career',
 'longest_career',
 'pass_plays_career',
 'receptions_career',
 'route_rate_career',
 'routes_career',
 'slot_rate_career',
 'slot_snaps_career',
 'targeted_qb_rating_career',
 'targets_receiving_career',
 'touchdowns_receiving_career',
 'wide_rate_career',
 'wide_snaps_career',
 'yards_receiving_career',
 'yards_after_catch_career',
 'grades_pass_block_career',
 'hits_allowed_career',
 'hurries_allowed_career',
 'snap_counts_offense_career',
 'non_spike_pass_block_career',
 'snap_counts_pass_block_career',
 'pressures_allowed_career',
 'sacks_allowed_career',
 'snap_counts_block_career',
 'snap_counts_run_block_career',
 'snap_counts_lt_career',
 'snap_counts_lg_career',
 'snap_counts_ce_career',
 'snap_counts_rg_career',
 'snap_counts_rt_career',
 'snap_counts_te_career',
 'assists_career',
 'batted_passes_career',
 'forced_fumbles_career',
 'fumble_recoveries_defensive_career',
 'fumble_recovery_touchdowns_career',
 'grades_coverage_defense_career',
 'grades_defense_career',
 'grades_defense_penalty_career',
 'grades_pass_rush_defense_career',
 'grades_run_defense_career',
 'grades_tackle_career',
 'hits_career',
 'hurries_career',
 'interception_touchdowns_career',
 'interceptions_defensive_career',
 'missed_tackle_rate_career',
 'missed_tackles_career',
 'pass_break_ups_career',
 'receptions_allowed_career',
 'sacks_defensive_career',
 'safeties_career',
 'stops_career',
 'tackles_career',
 'tackles_for_loss_career',
 'targets_allowed_career',
 'total_pressures_career',
 'coverage_touchdowns_allowed_career',
 'coverage_yards_allowed_career',
 'offensive/defensive_grade_career',
 'yards_passing/att_career',
 'completion_pct_career',
 'adj_completion_pct_career',
 'touchdown_pct_career',
 'interception_pct_career',
 'dangerous_play_pct_career',
 'money_throw_pct_career',
 '1st_downs/pass_att_career',
 'breakaway_runs/att_career',
 'fumbles/att_career',
 'touchdowns_rushing/att_career',
 'yards_rushing/att_career',
 'catch_pct_career',
 'contested_catch_pct_career',
 'touchdowns_receiving/target_career',
 'interceptions_receiving/target_career',
 'targets/route_ran_career',
 '1st_downs/route_ran_career',
 'drops/target_career',
 'yards/reception_career',
 'yards/target_career',
 'yards/route_ran_career',
 'yards_after_catch/reception_career',
 'yards_after_catch/target_career',
 'yards_after_catch/route_ran_career',
 'wide_snaps_routes_pct_career',
 'slot_snaps_routes_pct_career',
 'pass_blocking_efficency',
 'blocking_pct_per play',
 'avoided_tackles/touches_career',
 'fumbles/touches_career',
 'allowed_catch_pct',
 'yards_per_catch_allowed',
 'player_id_epa_career',
 'Season_epa_career',
 'CountablePlays_career',
 'TotalPPA All_career',
 'TotalPPA Pass_career',
 'TotalPPA Rush_career',
 'TotalPPA FirstDown_career',
 'TotalPPA SecondDown_career',
 'TotalPPA ThirdDown_career',
 'TotalPPA StandardDowns_career',
 'TotalPPA PassingDowns_career',
 'power_5_team',
 'Total_EPA_avg',
 'EPA_Pass_avg',
 'EPA_Rush_avg',
 'EPA_1st_down_avg',
 'EPA_2nd_down_avg',
 'EPA_3rd_down_avg',
 'EPA_StandardDowns_avg',
 'EPA_PassingDowns_avg',
 'Season_final_season',
 'player_game_count_final_season',
 'accuracy_percent',
 'aimed_passes_final_season',
 'attempts_passing_final_season',
 'avg_depth_of_target_passing_final_season',
 'avg_time_to_throw_final_season',
 'bats_final_season',
 'big_time_throws_final_season',
 'btt_rate',
 'completion_percent',
 'completions_final_season',
 'declined_penalties_final_season',
 'def_gen_pressures_final_season',
 'drop_rate',
 'dropbacks_final_season',
 'drops_passing_final_season',
 'first_downs_passing_final_season',
 'franchise_id',
 'grades_hands_fumble_final_season',
 'grades_offense_final_season',
 'grades_pass_final_season',
 'grades_run_final_season',
 'hit_as_threw_final_season',
 'interceptions_passing_final_season',
 'passing_snaps_final_season',
 'penalties_final_season',
 'pressure_to_sack_rate_final_season',
 'qb_rating_final_season',
 'sack_percent',
 'sacks_final_season',
 'scrambles_final_season',
 'spikes_final_season',
 'thrown_aways_final_season',
 'touchdowns_passing_final_season',
 'turnover_worthy_plays_final_season',
 'twp_rate',
 'yards_passing_final_season',
 'ypa_final_season',
 'attempts_rushing_final_season',
 'avoided_tackles_final_season',
 'breakaway_attempts_final_season',
 'breakaway_percent',
 'breakaway_yards_final_season',
 'designed_yards_final_season',
 'drops',
 'elu_recv_mtf_final_season',
 'elu_rush_mtf_final_season',
 'elu_yco_final_season',
 'elusive_rating_final_season',
 'explosive_final_season',
 'first_downs_rushing_final_season',
 'fumbles_final_season',
 'gap_attempts_final_season',
 'grades_offense_penalty_final_season',
 'grades_pass_block_final_season',
 'grades_pass_route_final_season',
 'grades_run_block_final_season',
 'longest_rush_final_season',
 'rec_yards',
 'receptions_final_season',
 'routes_final_season',
 'run_plays_final_season',
 'scramble_yards_final_season',
 'targets_1st_year_stats',
 'total_touches_final_season',
 'touchdowns_rushing_final_season',
 'yards_rushing_final_season',
 'yards_after_contact_final_season',
 'yco_attempt_final_season',
 'yprr',
 'zone_attempts_final_season',
 'avg_depth_of_target_receiving_final_season',
 'caught_percent',
 'contested_catch_rate',
 'contested_receptions_final_season',
 'contested_targets_final_season',
 'drops_receiving_final_season',
 'first_downs_receiving_final_season',
 'grades_hands_drop_final_season',
 'inline_rate_final_season',
 'inline_snaps_final_season',
 'interceptions_receiving_final_season',
 'longest_final_season',
 'pass_block_rate',
 'pass_blocks',
 'pass_plays_final_season',
 'route_rate_final_season',
 'slot_rate_final_season',
 'slot_snaps_final_season',
 'targeted_qb_rating_final_season',
 'targets_receiving_final_season',
 'touchdowns_receiving_final_season',
 'wide_rate_final_season',
 'wide_snaps_final_season',
 'yards_receiving_final_season',
 'yards_after_catch_final_season',
 'yards_after_catch_per_reception',
 'yards_per_reception',
 'block_percent',
 'hits_allowed_final_season',
 'hurries_allowed_final_season',
 'non_spike_pass_block_final_season',
 'non_spike_pass_block_percentage',
 'pass_block_percent',
 'pbe',
 'pressures_allowed_final_season',
 'sacks_allowed_final_season',
 'snap_counts_block_final_season',
 'snap_counts_ce_final_season',
 'snap_counts_lg_final_season',
 'snap_counts_lt_final_season',
 'snap_counts_offense_final_season',
 'snap_counts_pass_block_final_season',
 'snap_counts_pass_play',
 'snap_counts_rg_final_season',
 'snap_counts_rt_final_season',
 'snap_counts_run_block_final_season',
 'snap_counts_te_final_season',
 'assists_final_season',
 'batted_passes_final_season',
 'catch_rate',
 'forced_fumbles_final_season',
 'fumble_recoveries_defensive_final_season',
 'fumble_recovery_touchdowns_final_season',
 'grades_coverage_defense_final_season',
 'grades_defense_final_season',
 'grades_defense_penalty_final_season',
 'grades_pass_rush_defense_final_season',
 'grades_run_defense_final_season',
 'grades_tackle_final_season',
 'hits_final_season',
 'hurries_final_season',
 'interception_touchdowns_final_season',
 'interceptions_defensive_final_season',
 'missed_tackle_rate_final_season',
 'missed_tackles_final_season',
 'pass_break_ups_final_season',
 'qb_rating_against',
 'receptions_allowed_final_season',
 'sacks_defensive_final_season',
 'safeties_final_season',
 'snap_counts_box',
 'snap_counts_corner',
 'snap_counts_coverage',
 'snap_counts_defense',
 'snap_counts_dl',
 'snap_counts_dl_a_gap',
 'snap_counts_dl_b_gap',
 'snap_counts_dl_outside_t',
 'snap_counts_dl_over_t',
 'snap_counts_fs',
 'snap_counts_offball',
 'snap_counts_pass_rush',
 'snap_counts_run_defense',
 'snap_counts_slot',
 'stops_final_season',
 'tackles_final_season',
 'tackles_for_loss_final_season',
 'targets_allowed_final_season',
 'total_pressures_final_season',
 'coverage_touchdowns_allowed_final_season',
 'coverage_yards_allowed_final_season',
 'offensive/defensive_grade_final_season',
 'player_id_epa_final_season',
 'Season_epa_final_season',
 'CountablePlays_final_season',
 'AveragePPA All',
 'AveragePPA Pass',
 'AveragePPA Rush',
 'AveragePPA FirstDown',
 'AveragePPA SecondDown',
 'AveragePPA ThirdDown',
 'AveragePPA StandardDowns',
 'AveragePPA PassingDowns',
 'TotalPPA All_final_season',
 'TotalPPA Pass_final_season',
 'TotalPPA Rush_final_season',
 'TotalPPA FirstDown_final_season',
 'TotalPPA SecondDown_final_season',
 'TotalPPA ThirdDown_final_season',
 'TotalPPA StandardDowns_final_season',
 'TotalPPA PassingDowns_final_season',
 'completions_1st_year_stats',
 'attempts_1st_year_stats',
 'passing_yards_1st_year_stats',
 'passing_tds_1st_year_stats',
 'interceptions_1st_year_stats',
 'sacks_1st_year_stats',
 'sack_yards_1st_year_stats',
 'sack_fumbles_1st_year_stats',
 'sack_fumbles_lost_1st_year_stats',
 'passing_air_yards_1st_year_stats',
 'passing_yards_after_catch_1st_year_stats',
 'passing_first_downs_1st_year_stats',
 'passing_epa_1st_year_stats',
 'passing_2pt_conversions_1st_year_stats',
 'pacr_1st_year_stats',
 'dakota_1st_year_stats',
 'carries_1st_year_stats',
 'rushing_yards_1st_year_stats',
 'rushing_tds_1st_year_stats',
 'rushing_fumbles_1st_year_stats',
 'rushing_fumbles_lost_1st_year_stats',
 'rushing_first_downs_1st_year_stats',
 'rushing_epa_1st_year_stats',
 'rushing_2pt_conversions_1st_year_stats',
 'receptions_1st_year_stats',
 'targets_y',
 'receiving_yards_1st_year_stats',
 'receiving_tds_1st_year_stats',
 'receiving_fumbles_1st_year_stats',
 'receiving_fumbles_lost_1st_year_stats',
 'receiving_air_yards_1st_year_stats',
 'receiving_yards_after_catch_1st_year_stats',
 'receiving_first_downs_1st_year_stats',
 'receiving_epa_1st_year_stats',
 'receiving_2pt_conversions_1st_year_stats',
 'racr_1st_year_stats',
 'target_share_1st_year_stats',
 'air_yards_share_1st_year_stats',
 'wopr_x_1st_year_stats',
 'special_teams_tds_1st_year_stats',
 'fantasy_points_1st_year_stats',
 'fantasy_points_ppr_1st_year_stats',
 'games_1st_year_stats',
 'tgt_sh_1st_year_stats',
 'ay_sh_1st_year_stats',
 'yac_sh_1st_year_stats',
 'wopr_y_1st_year_stats',
 'ry_sh_1st_year_stats',
 'rtd_sh_1st_year_stats',
 'rfd_sh_1st_year_stats',
 'rtdfd_sh_1st_year_stats',
 'dom_1st_year_stats',
 'w8dom_1st_year_stats',
 'yptmpa_1st_year_stats',
 'ppr_sh_1st_year_stats',
 'season_defensive',
 'player_game_count',
 'tackles',
 'assisted_tackles',
 'tackles_for_loss',
 'stops',
 'missed_tackle_rate',
 'missed_tackles',
 'forced_fumbles',
 'fumble_recoveries',
 'fumble_recovery_touchdowns',
 'total_defensive_pressures',
 'pass_rushing_hurries',
 'hits',
 'defensive_sacks',
 'safeties',
 'defensive_interceptions',
 'defensive_interception_touchdowns',
 'pass_break_ups',
 'coverage_targets',
 'allowed_recpetions',
 'allowed_passing_yards',
 'allowed_passing_touchdowns']

In [232]:
#Limiting to offensive skill positions, at the moment
pos_list = ['QB', 'RB', 'WR', 'TE']

In [233]:
export_df = pd.DataFrame()

for pos in pos_list:
    print(pos)
    temp_df = second_year_df[second_year_df['pos'] == pos]
    temp_df = temp_df[temp_df['2nd_season'] < current_year]
    pos_df = second_year_df[(second_year_df['pos'] == pos) & (second_year_df['2nd_season'] < current_year + 1)]
    
    for stat in stats_to_perdict:
        print(stat)
        X = temp_df[x_stats]
        Y = temp_df[stat]

        gbr_model = GradientBoostingRegressor(n_estimators=(temp_df.shape[0]), learning_rate=0.1)
        gbr_model.fit(X, Y)
        gbr_predictions = gbr_model.predict(pos_df[x_stats])
        pos_df[stat + '_y_pred_gbf'] = gbr_predictions
    export_df = pd.concat([export_df,pos_df])
        
export_df

QB
games_2nd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\21534768.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


passing_yards_2nd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\21534768.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


passing_tds_2nd_year_stats


...

dom_2nd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\21534768.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


yptmpa_2nd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\21534768.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,special_teams_tds_2nd_year_stats_y_pred_gbf,tgt_sh_2nd_year_stats_y_pred_gbf,ay_sh_2nd_year_stats_y_pred_gbf,yac_sh_2nd_year_stats_y_pred_gbf,wopr_y_2nd_year_stats_y_pred_gbf,ry_sh_2nd_year_stats_y_pred_gbf,rtd_sh_2nd_year_stats_y_pred_gbf,rfd_sh_2nd_year_stats_y_pred_gbf,dom_2nd_year_stats_y_pred_gbf,yptmpa_2nd_year_stats_y_pred_gbf
12,402.0,155.0,2015.0,brett-hundley-1,Brett Hundley,QB,UCLA,75.0,226.0,4.63,...,0.0,-0.000006,2.407841e-07,5.719253e-07,5.025389e-07,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
13,432.0,191.0,2015.0,sean-mannion-1,Sean Mannion,QB,Oregon State,78.0,229.0,5.14,...,0.0,-0.000003,2.846089e-07,5.719253e-07,-4.908184e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
14,433.0,192.0,2015.0,marcus-mariota-1,Marcus Mariota,QB,Oregon,76.0,222.0,4.52,...,0.0,0.002065,3.947706e-03,5.719253e-07,6.272463e-03,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
15,462.0,229.0,2015.0,bryce-petty-1,Bryce Petty,QB,Baylor,75.0,230.0,4.87,...,0.0,-0.000005,2.846089e-07,5.719253e-07,-4.908184e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
16,527.0,305.0,2015.0,jameis-winston-1,Jameis Winston,QB,Florida State,76.0,231.0,4.97,...,0.0,-0.000005,2.846089e-07,5.719253e-07,-4.521846e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,2871.0,2830.0,2023.0,brenton-strange-1,Brenton Strange,TE,Penn St.,76.0,253.0,4.75,...,0.0,0.068385,2.457607e-02,9.747611e-02,1.304862e-01,6.876139e-02,0.028166,8.029974e-02,0.053821,0.444813
542,2853.0,2810.0,2023.0,luke-schoonmaker-1,Luke Schoonmaker,TE,Michigan,77.0,251.0,4.75,...,0.0,0.047969,4.508032e-02,7.986247e-02,1.173480e-01,6.579088e-02,0.012192,6.125382e-02,0.034468,0.403883
547,2653.0,2579.0,2023.0,davis-allen-1,Davis Allen,TE,Clemson,78.0,245.0,4.75,...,0.0,0.073115,6.166197e-02,8.365349e-02,1.432403e-01,8.420760e-02,0.064412,1.003118e-01,0.075454,0.677898
559,2800.0,2751.0,2023.0,michael-mayer-1,Michael Mayer,TE,Notre Dame,77.0,249.0,4.75,...,0.0,0.104413,9.238106e-02,1.304535e-01,2.203628e-01,1.368338e-01,0.206162,1.277500e-01,0.143609,1.008864


In [234]:
export_df.fillna(0)
#export_df.to_csv('export.csv')

,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,special_teams_tds_2nd_year_stats_y_pred_gbf,tgt_sh_2nd_year_stats_y_pred_gbf,ay_sh_2nd_year_stats_y_pred_gbf,yac_sh_2nd_year_stats_y_pred_gbf,wopr_y_2nd_year_stats_y_pred_gbf,ry_sh_2nd_year_stats_y_pred_gbf,rtd_sh_2nd_year_stats_y_pred_gbf,rfd_sh_2nd_year_stats_y_pred_gbf,dom_2nd_year_stats_y_pred_gbf,yptmpa_2nd_year_stats_y_pred_gbf
12,402.0,155.0,2015.0,brett-hundley-1,Brett Hundley,QB,UCLA,75.0,226.0,4.63,...,0.0,-0.000006,2.407841e-07,5.719253e-07,5.025389e-07,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
13,432.0,191.0,2015.0,sean-mannion-1,Sean Mannion,QB,Oregon State,78.0,229.0,5.14,...,0.0,-0.000003,2.846089e-07,5.719253e-07,-4.908184e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
14,433.0,192.0,2015.0,marcus-mariota-1,Marcus Mariota,QB,Oregon,76.0,222.0,4.52,...,0.0,0.002065,3.947706e-03,5.719253e-07,6.272463e-03,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
15,462.0,229.0,2015.0,bryce-petty-1,Bryce Petty,QB,Baylor,75.0,230.0,4.87,...,0.0,-0.000005,2.846089e-07,5.719253e-07,-4.908184e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
16,527.0,305.0,2015.0,jameis-winston-1,Jameis Winston,QB,Florida State,76.0,231.0,4.97,...,0.0,-0.000005,2.846089e-07,5.719253e-07,-4.521846e-06,2.580232e-07,0.000002,3.567552e-07,0.000001,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,2871.0,2830.0,2023.0,brenton-strange-1,Brenton Strange,TE,Penn St.,76.0,253.0,4.75,...,0.0,0.068385,2.457607e-02,9.747611e-02,1.304862e-01,6.876139e-02,0.028166,8.029974e-02,0.053821,0.444813
542,2853.0,2810.0,2023.0,luke-schoonmaker-1,Luke Schoonmaker,TE,Michigan,77.0,251.0,4.75,...,0.0,0.047969,4.508032e-02,7.986247e-02,1.173480e-01,6.579088e-02,0.012192,6.125382e-02,0.034468,0.403883
547,2653.0,2579.0,2023.0,davis-allen-1,Davis Allen,TE,Clemson,78.0,245.0,4.75,...,0.0,0.073115,6.166197e-02,8.365349e-02,1.432403e-01,8.420760e-02,0.064412,1.003118e-01,0.075454,0.677898
559,2800.0,2751.0,2023.0,michael-mayer-1,Michael Mayer,TE,Notre Dame,77.0,249.0,4.75,...,0.0,0.104413,9.238106e-02,1.304535e-01,2.203628e-01,1.368338e-01,0.206162,1.277500e-01,0.143609,1.008864


In [235]:
filter_columns = ['player_name','pos','school','2nd_season']

perdicted_stats = [item + '_y_pred_gbf' for item in stats_to_perdict]

filtered_columns = filter_columns + perdicted_stats

In [236]:
filtered_export = export_df[filtered_columns]

filtered_export['scrimmage_yards'] = filtered_export['rushing_yards_2nd_year_stats_y_pred_gbf'] + filtered_export['receiving_yards_2nd_year_stats_y_pred_gbf']
filtered_export['scrimmage_TDs'] = filtered_export['rushing_tds_2nd_year_stats_y_pred_gbf'] + filtered_export['receiving_tds_2nd_year_stats_y_pred_gbf']

scoring_dict = {
'passing_yards_2nd_year_stats_y_pred_gbf': .04,
'passing_tds_2nd_year_stats_y_pred_gbf': 4,
'interceptions_2nd_year_stats_y_pred_gbf' : -2,
'rushing_yards_2nd_year_stats_y_pred_gbf' : .1,
'rushing_tds_2nd_year_stats_y_pred_gbf' : 6,
'rushing_fumbles_2nd_year_stats_y_pred_gbf' : -1,
'rushing_fumbles_lost_2nd_year_stats_y_pred_gbf' : -1,
'rushing_first_downs_2nd_year_stats_y_pred_gbf': .5,
'receptions_2nd_year_stats_y_pred_gbf' : .5,
'receiving_yards_2nd_year_stats_y_pred_gbf' : .1,
'receiving_tds_2nd_year_stats_y_pred_gbf' : 6,
'receiving_fumbles_2nd_year_stats_y_pred_gbf' : -1,
'receiving_fumbles_lost_2nd_year_stats_y_pred_gbf' : -1,
'receiving_first_downs_2nd_year_stats_y_pred_gbf' : .5
}

filtered_export['fantasy_points'] = filtered_export[scoring_dict.keys()].mul(list(scoring_dict.values()), axis=1).sum(axis=1)
filtered_export['fantasy_points/G'] = filtered_export['fantasy_points'] / filtered_export['games_2nd_year_stats_y_pred_gbf']

filtered_export = filtered_export.sort_values(by='fantasy_points/G', ascending=False)

filtered_export.to_csv('filtered_2nd_year_prediction.csv')

C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\2618261176.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_export['scrimmage_yards'] = filtered_export['rushing_yards_2nd_year_stats_y_pred_gbf'] + filtered_export['receiving_yards_2nd_year_stats_y_pred_gbf']
C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\2618261176.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_export['scrimmage_TDs'] = filtered_export['rushing_tds_2nd_year_stats_y_pred_gbf'] + filtered_export['receiving_td

# Creating 3rd Year Projections

In [237]:
third_year_df = second_year_df.merge(seasonal_data_df, left_on='player_id_3rd_season', right_on='player_id_season', how='left')
third_year_df = third_year_df.fillna(0)
third_year_df = third_year_df[(third_year_df['player_name'] != 0) & (third_year_df['3rd_season'] < current_year+1)]
third_year_df

,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,player_id_season_y
0,0.0,0.0,2015.0,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.60,...,0.117387,0.042937,0.037037,0.072222,0.067633,0.039987,0.041757,0.329268,0.100143,00-0032104_2017
1,354.0,105.0,2015.0,devin-funchess-1,Devin Funchess,WR,Michigan,76.0,232.0,4.70,...,0.603898,0.253088,0.363636,0.239521,0.253968,0.308362,0.275198,1.676647,0.156017,00-0032055_2017
2,335.0,83.0,2015.0,phillip-dorsett-1,Phillip Dorsett,WR,Miami (FL),70.0,185.0,4.33,...,0.175403,0.076832,0.000000,0.062016,0.054054,0.038416,0.061465,0.595092,0.034120,00-0032208_2017
3,332.0,80.0,2015.0,stefon-diggs-1,Stefon Diggs,WR,Maryland,72.0,195.0,4.46,...,0.555662,0.242433,0.333333,0.244186,0.255102,0.287883,0.260613,1.853712,0.155710,00-0031588_2017
4,324.0,70.0,2015.0,geremy-davis-1,Geremy Davis,WR,Connecticut,74.0,216.0,4.49,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2562.0,2476.0,2022.0,teagan-quitoriano-1,Teagan Quitoriano,TE,Oregon St.,78.0,258.0,4.75,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
512,2579.0,2493.0,2022.0,jeremy-ruckert-1,Jeremy Ruckert,TE,Ohio St.,77.0,250.0,4.75,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
513,2613.0,2531.0,2022.0,cole-turner-2,Cole Turner,TE,Nevada,78.0,246.0,4.76,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
514,2638.0,2563.0,2022.0,jelani-woods-1,Jelani Woods,TE,Virginia,79.0,259.0,4.61,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [238]:
named_columns = ['season',
'season_type',
'completions',
'attempts',
'passing_yards',
'passing_tds',
'interceptions',
'sacks',
'sack_yards',
'sack_fumbles',
'sack_fumbles_lost',
'passing_air_yards',
'passing_yards_after_catch',
'passing_first_downs',
'passing_epa',
'passing_2pt_conversions',
'pacr',
'dakota',
'carries',
'rushing_yards',
'rushing_tds',
'rushing_fumbles',
'rushing_fumbles_lost',
'rushing_first_downs',
'rushing_epa',
'rushing_2pt_conversions',
'receptions',
'targets',
'receiving_yards',
'receiving_tds',
'receiving_fumbles',
'receiving_fumbles_lost',
'receiving_air_yards',
'receiving_yards_after_catch',
'receiving_first_downs',
'receiving_epa',
'receiving_2pt_conversions',
'racr',
'target_share',
'air_yards_share',
'wopr_x',
'special_teams_tds',
'fantasy_points',
'fantasy_points_ppr',
'games',
'tgt_sh',
'ay_sh',
'yac_sh',
'wopr_y',
'ry_sh',
'rtd_sh',
'rfd_sh',
'rtdfd_sh',
'dom',
'w8dom',
'yptmpa',
'ppr_sh']

renamed_columns = [col + '_3rd_year_stats' for col in named_columns]

In [239]:
column_dict = {}

column_dict = dict(zip(named_columns,renamed_columns))

column_dict

{'season': 'season_3rd_year_stats',
 'season_type': 'season_type_3rd_year_stats',
 'completions': 'completions_3rd_year_stats',
 'attempts': 'attempts_3rd_year_stats',
 'passing_yards': 'passing_yards_3rd_year_stats',
 'passing_tds': 'passing_tds_3rd_year_stats',
 'interceptions': 'interceptions_3rd_year_stats',
 'sacks': 'sacks_3rd_year_stats',
 'sack_yards': 'sack_yards_3rd_year_stats',
 'sack_fumbles': 'sack_fumbles_3rd_year_stats',
 'sack_fumbles_lost': 'sack_fumbles_lost_3rd_year_stats',
 'passing_air_yards': 'passing_air_yards_3rd_year_stats',
 'passing_yards_after_catch': 'passing_yards_after_catch_3rd_year_stats',
 'passing_first_downs': 'passing_first_downs_3rd_year_stats',
 'passing_epa': 'passing_epa_3rd_year_stats',
 'passing_2pt_conversions': 'passing_2pt_conversions_3rd_year_stats',
 'pacr': 'pacr_3rd_year_stats',
 'dakota': 'dakota_3rd_year_stats',
 'carries': 'carries_3rd_year_stats',
 'rushing_yards': 'rushing_yards_3rd_year_stats',
 'rushing_tds': 'rushing_tds_3rd_yea

In [240]:
third_year_df = third_year_df.rename(columns=column_dict)

#third_year_df.to_csv('3rd_year_raw.csv')

third_year_df

,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,wopr_y_3rd_year_stats,ry_sh_3rd_year_stats,rtd_sh_3rd_year_stats,rfd_sh_3rd_year_stats,rtdfd_sh_3rd_year_stats,dom_3rd_year_stats,w8dom_3rd_year_stats,yptmpa_3rd_year_stats,ppr_sh_3rd_year_stats,player_id_season_y
0,0.0,0.0,2015.0,ameer-abdullah-1,Ameer Abdullah,RB,Nebraska,69.0,205.0,4.60,...,0.117387,0.042937,0.037037,0.072222,0.067633,0.039987,0.041757,0.329268,0.100143,00-0032104_2017
1,354.0,105.0,2015.0,devin-funchess-1,Devin Funchess,WR,Michigan,76.0,232.0,4.70,...,0.603898,0.253088,0.363636,0.239521,0.253968,0.308362,0.275198,1.676647,0.156017,00-0032055_2017
2,335.0,83.0,2015.0,phillip-dorsett-1,Phillip Dorsett,WR,Miami (FL),70.0,185.0,4.33,...,0.175403,0.076832,0.000000,0.062016,0.054054,0.038416,0.061465,0.595092,0.034120,00-0032208_2017
3,332.0,80.0,2015.0,stefon-diggs-1,Stefon Diggs,WR,Maryland,72.0,195.0,4.46,...,0.555662,0.242433,0.333333,0.244186,0.255102,0.287883,0.260613,1.853712,0.155710,00-0031588_2017
4,324.0,70.0,2015.0,geremy-davis-1,Geremy Davis,WR,Connecticut,74.0,216.0,4.49,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2562.0,2476.0,2022.0,teagan-quitoriano-1,Teagan Quitoriano,TE,Oregon St.,78.0,258.0,4.75,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
512,2579.0,2493.0,2022.0,jeremy-ruckert-1,Jeremy Ruckert,TE,Ohio St.,77.0,250.0,4.75,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
513,2613.0,2531.0,2022.0,cole-turner-2,Cole Turner,TE,Nevada,78.0,246.0,4.76,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
514,2638.0,2563.0,2022.0,jelani-woods-1,Jelani Woods,TE,Virginia,79.0,259.0,4.61,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [241]:
stats_to_perdict = [ 'games_3rd_year_stats',
 'passing_yards_3rd_year_stats',
 'passing_tds_3rd_year_stats',
 'interceptions_3rd_year_stats',
 'sacks_3rd_year_stats',
 'sack_yards_3rd_year_stats',
 'passing_air_yards_3rd_year_stats',
 'passing_yards_after_catch_3rd_year_stats',
 'passing_first_downs_3rd_year_stats',
 'passing_epa_3rd_year_stats',
 'pacr_3rd_year_stats',
 'dakota_3rd_year_stats',
 'rushing_yards_3rd_year_stats',
 'rushing_tds_3rd_year_stats',
 'rushing_fumbles_3rd_year_stats',
 'rushing_fumbles_lost_3rd_year_stats',
 'rushing_first_downs_3rd_year_stats',
 'rushing_epa_3rd_year_stats',
 'targets_3rd_year_stats',
 'receptions_3rd_year_stats',
 'receiving_yards_3rd_year_stats',
 'receiving_tds_3rd_year_stats',
 'receiving_fumbles_3rd_year_stats',
 'receiving_fumbles_lost_3rd_year_stats',
 'receiving_air_yards_3rd_year_stats',
 'receiving_yards_after_catch_3rd_year_stats',
 'receiving_first_downs_3rd_year_stats',
 'receiving_epa_3rd_year_stats',
 'racr_3rd_year_stats',
 'air_yards_share_3rd_year_stats',
 'wopr_x_3rd_year_stats',
 'special_teams_tds_3rd_year_stats',
 'tgt_sh_3rd_year_stats',
 'ay_sh_3rd_year_stats',
 'yac_sh_3rd_year_stats',
 'wopr_y_3rd_year_stats',
 'ry_sh_3rd_year_stats',
 'rtd_sh_3rd_year_stats',
 'rfd_sh_3rd_year_stats',
 'dom_3rd_year_stats',
 'yptmpa_3rd_year_stats']

x_stats = ['ht',
 'wt',
 'forty',
 'bench',
 'vertical',
 'broad_jump',
 'cone',
 'shuttle',
 'draft_ovr',
 'draft_round',
 'Season_career',
 'player_game_count_career',
 'aimed_passes_career',
 'attempts_passing_career',
 'avg_depth_of_target_passing_career',
 'avg_time_to_throw_career',
 'bats_career',
 'big_time_throws_career',
 'completions_career',
 'declined_penalties_career',
 'def_gen_pressures_career',
 'dropbacks_career',
 'drops_passing_career',
 'first_downs_passing_career',
 'grades_hands_fumble_career',
 'grades_offense_career',
 'grades_pass_career',
 'hit_as_threw_career',
 'interceptions_passing_career',
 'passing_snaps_career',
 'penalties_career',
 'pressure_to_sack_rate_career',
 'qb_rating_career',
 'sacks_career',
 'scrambles_career',
 'spikes_career',
 'thrown_aways_career',
 'touchdowns_passing_career',
 'turnover_worthy_plays_career',
 'yards_passing_career',
 'attempts_rushing_career',
 'avoided_tackles_career',
 'breakaway_attempts_career',
 'breakaway_yards_career',
 'designed_yards_career',
 'elu_recv_mtf_career',
 'elu_rush_mtf_career',
 'elu_yco_career',
 'elusive_rating_career',
 'explosive_career',
 'first_downs_rushing_career',
 'fumbles_career',
 'gap_attempts_career',
 'grades_run_career',
 'grades_offense_penalty_career',
 'grades_run_block_career',
 'longest_rush_career',
 'run_plays_career',
 'scramble_yards_career',
 'total_touches_career',
 'touchdowns_rushing_career',
 'yards_rushing_career',
 'yards_after_contact_career',
 'yco_attempt_career',
 'ypa_career',
 'zone_attempts_career',
 'avg_depth_of_target_receiving_career',
 'contested_receptions_career',
 'contested_targets_career',
 'drops_receiving_career',
 'first_downs_receiving_career',
 'grades_hands_drop_career',
 'grades_pass_route_career',
 'inline_rate_career',
 'inline_snaps_career',
 'interceptions_receiving_career',
 'longest_career',
 'pass_plays_career',
 'receptions_career',
 'route_rate_career',
 'routes_career',
 'slot_rate_career',
 'slot_snaps_career',
 'targeted_qb_rating_career',
 'targets_receiving_career',
 'touchdowns_receiving_career',
 'wide_rate_career',
 'wide_snaps_career',
 'yards_receiving_career',
 'yards_after_catch_career',
 'grades_pass_block_career',
 'hits_allowed_career',
 'hurries_allowed_career',
 'snap_counts_offense_career',
 'non_spike_pass_block_career',
 'snap_counts_pass_block_career',
 'pressures_allowed_career',
 'sacks_allowed_career',
 'snap_counts_block_career',
 'snap_counts_run_block_career',
 'snap_counts_lt_career',
 'snap_counts_lg_career',
 'snap_counts_ce_career',
 'snap_counts_rg_career',
 'snap_counts_rt_career',
 'snap_counts_te_career',
 'assists_career',
 'batted_passes_career',
 'forced_fumbles_career',
 'fumble_recoveries_defensive_career',
 'fumble_recovery_touchdowns_career',
 'grades_coverage_defense_career',
 'grades_defense_career',
 'grades_defense_penalty_career',
 'grades_pass_rush_defense_career',
 'grades_run_defense_career',
 'grades_tackle_career',
 'hits_career',
 'hurries_career',
 'interception_touchdowns_career',
 'interceptions_defensive_career',
 'missed_tackle_rate_career',
 'missed_tackles_career',
 'pass_break_ups_career',
 'receptions_allowed_career',
 'sacks_defensive_career',
 'safeties_career',
 'stops_career',
 'tackles_career',
 'tackles_for_loss_career',
 'targets_allowed_career',
 'total_pressures_career',
 'coverage_touchdowns_allowed_career',
 'coverage_yards_allowed_career',
 'offensive/defensive_grade_career',
 'yards_passing/att_career',
 'completion_pct_career',
 'adj_completion_pct_career',
 'touchdown_pct_career',
 'interception_pct_career',
 'dangerous_play_pct_career',
 'money_throw_pct_career',
 '1st_downs/pass_att_career',
 'breakaway_runs/att_career',
 'fumbles/att_career',
 'touchdowns_rushing/att_career',
 'yards_rushing/att_career',
 'catch_pct_career',
 'contested_catch_pct_career',
 'touchdowns_receiving/target_career',
 'interceptions_receiving/target_career',
 'targets/route_ran_career',
 '1st_downs/route_ran_career',
 'drops/target_career',
 'yards/reception_career',
 'yards/target_career',
 'yards/route_ran_career',
 'yards_after_catch/reception_career',
 'yards_after_catch/target_career',
 'yards_after_catch/route_ran_career',
 'wide_snaps_routes_pct_career',
 'slot_snaps_routes_pct_career',
 'pass_blocking_efficency',
 'blocking_pct_per play',
 'avoided_tackles/touches_career',
 'fumbles/touches_career',
 'allowed_catch_pct',
 'yards_per_catch_allowed',
 'player_id_epa_career',
 'Season_epa_career',
 'CountablePlays_career',
 'TotalPPA All_career',
 'TotalPPA Pass_career',
 'TotalPPA Rush_career',
 'TotalPPA FirstDown_career',
 'TotalPPA SecondDown_career',
 'TotalPPA ThirdDown_career',
 'TotalPPA StandardDowns_career',
 'TotalPPA PassingDowns_career',
 'power_5_team',
 'Total_EPA_avg',
 'EPA_Pass_avg',
 'EPA_Rush_avg',
 'EPA_1st_down_avg',
 'EPA_2nd_down_avg',
 'EPA_3rd_down_avg',
 'EPA_StandardDowns_avg',
 'EPA_PassingDowns_avg',
 'Season_final_season',
 'player_game_count_final_season',
 'accuracy_percent',
 'aimed_passes_final_season',
 'attempts_passing_final_season',
 'avg_depth_of_target_passing_final_season',
 'avg_time_to_throw_final_season',
 'bats_final_season',
 'big_time_throws_final_season',
 'btt_rate',
 'completion_percent',
 'completions_final_season',
 'declined_penalties_final_season',
 'def_gen_pressures_final_season',
 'drop_rate',
 'dropbacks_final_season',
 'drops_passing_final_season',
 'first_downs_passing_final_season',
 'franchise_id',
 'grades_hands_fumble_final_season',
 'grades_offense_final_season',
 'grades_pass_final_season',
 'grades_run_final_season',
 'hit_as_threw_final_season',
 'interceptions_passing_final_season',
 'passing_snaps_final_season',
 'penalties_final_season',
 'pressure_to_sack_rate_final_season',
 'qb_rating_final_season',
 'sack_percent',
 'sacks_final_season',
 'scrambles_final_season',
 'spikes_final_season',
 'thrown_aways_final_season',
 'touchdowns_passing_final_season',
 'turnover_worthy_plays_final_season',
 'twp_rate',
 'yards_passing_final_season',
 'ypa_final_season',
 'attempts_rushing_final_season',
 'avoided_tackles_final_season',
 'breakaway_attempts_final_season',
 'breakaway_percent',
 'breakaway_yards_final_season',
 'designed_yards_final_season',
 'drops',
 'elu_recv_mtf_final_season',
 'elu_rush_mtf_final_season',
 'elu_yco_final_season',
 'elusive_rating_final_season',
 'explosive_final_season',
 'first_downs_rushing_final_season',
 'fumbles_final_season',
 'gap_attempts_final_season',
 'grades_offense_penalty_final_season',
 'grades_pass_block_final_season',
 'grades_pass_route_final_season',
 'grades_run_block_final_season',
 'longest_rush_final_season',
 'rec_yards',
 'receptions_final_season',
 'routes_final_season',
 'run_plays_final_season',
 'scramble_yards_final_season',
 'targets_1st_year_stats',
 'total_touches_final_season',
 'touchdowns_rushing_final_season',
 'yards_rushing_final_season',
 'yards_after_contact_final_season',
 'yco_attempt_final_season',
 'yprr',
 'zone_attempts_final_season',
 'avg_depth_of_target_receiving_final_season',
 'caught_percent',
 'contested_catch_rate',
 'contested_receptions_final_season',
 'contested_targets_final_season',
 'drops_receiving_final_season',
 'first_downs_receiving_final_season',
 'grades_hands_drop_final_season',
 'inline_rate_final_season',
 'inline_snaps_final_season',
 'interceptions_receiving_final_season',
 'longest_final_season',
 'pass_block_rate',
 'pass_blocks',
 'pass_plays_final_season',
 'route_rate_final_season',
 'slot_rate_final_season',
 'slot_snaps_final_season',
 'targeted_qb_rating_final_season',
 'targets_receiving_final_season',
 'touchdowns_receiving_final_season',
 'wide_rate_final_season',
 'wide_snaps_final_season',
 'yards_receiving_final_season',
 'yards_after_catch_final_season',
 'yards_after_catch_per_reception',
 'yards_per_reception',
 'block_percent',
 'hits_allowed_final_season',
 'hurries_allowed_final_season',
 'non_spike_pass_block_final_season',
 'non_spike_pass_block_percentage',
 'pass_block_percent',
 'pbe',
 'pressures_allowed_final_season',
 'sacks_allowed_final_season',
 'snap_counts_block_final_season',
 'snap_counts_ce_final_season',
 'snap_counts_lg_final_season',
 'snap_counts_lt_final_season',
 'snap_counts_offense_final_season',
 'snap_counts_pass_block_final_season',
 'snap_counts_pass_play',
 'snap_counts_rg_final_season',
 'snap_counts_rt_final_season',
 'snap_counts_run_block_final_season',
 'snap_counts_te_final_season',
 'assists_final_season',
 'batted_passes_final_season',
 'catch_rate',
 'forced_fumbles_final_season',
 'fumble_recoveries_defensive_final_season',
 'fumble_recovery_touchdowns_final_season',
 'grades_coverage_defense_final_season',
 'grades_defense_final_season',
 'grades_defense_penalty_final_season',
 'grades_pass_rush_defense_final_season',
 'grades_run_defense_final_season',
 'grades_tackle_final_season',
 'hits_final_season',
 'hurries_final_season',
 'interception_touchdowns_final_season',
 'interceptions_defensive_final_season',
 'missed_tackle_rate_final_season',
 'missed_tackles_final_season',
 'pass_break_ups_final_season',
 'qb_rating_against',
 'receptions_allowed_final_season',
 'sacks_defensive_final_season',
 'safeties_final_season',
 'snap_counts_box',
 'snap_counts_corner',
 'snap_counts_coverage',
 'snap_counts_defense',
 'snap_counts_dl',
 'snap_counts_dl_a_gap',
 'snap_counts_dl_b_gap',
 'snap_counts_dl_outside_t',
 'snap_counts_dl_over_t',
 'snap_counts_fs',
 'snap_counts_offball',
 'snap_counts_pass_rush',
 'snap_counts_run_defense',
 'snap_counts_slot',
 'stops_final_season',
 'tackles_final_season',
 'tackles_for_loss_final_season',
 'targets_allowed_final_season',
 'total_pressures_final_season',
 'coverage_touchdowns_allowed_final_season',
 'coverage_yards_allowed_final_season',
 'offensive/defensive_grade_final_season',
 'player_id_epa_final_season',
 'Season_epa_final_season',
 'CountablePlays_final_season',
 'AveragePPA All',
 'AveragePPA Pass',
 'AveragePPA Rush',
 'AveragePPA FirstDown',
 'AveragePPA SecondDown',
 'AveragePPA ThirdDown',
 'AveragePPA StandardDowns',
 'AveragePPA PassingDowns',
 'TotalPPA All_final_season',
 'TotalPPA Pass_final_season',
 'TotalPPA Rush_final_season',
 'TotalPPA FirstDown_final_season',
 'TotalPPA SecondDown_final_season',
 'TotalPPA ThirdDown_final_season',
 'TotalPPA StandardDowns_final_season',
 'TotalPPA PassingDowns_final_season',
 'completions_1st_year_stats',
 'attempts_1st_year_stats',
 'passing_yards_1st_year_stats',
 'passing_tds_1st_year_stats',
 'interceptions_1st_year_stats',
 'sacks_1st_year_stats',
 'sack_yards_1st_year_stats',
 'sack_fumbles_1st_year_stats',
 'sack_fumbles_lost_1st_year_stats',
 'passing_air_yards_1st_year_stats',
 'passing_yards_after_catch_1st_year_stats',
 'passing_first_downs_1st_year_stats',
 'passing_epa_1st_year_stats',
 'passing_2pt_conversions_1st_year_stats',
 'pacr_1st_year_stats',
 'dakota_1st_year_stats',
 'carries_1st_year_stats',
 'rushing_yards_1st_year_stats',
 'rushing_tds_1st_year_stats',
 'rushing_fumbles_1st_year_stats',
 'rushing_fumbles_lost_1st_year_stats',
 'rushing_first_downs_1st_year_stats',
 'rushing_epa_1st_year_stats',
 'rushing_2pt_conversions_1st_year_stats',
 'receptions_1st_year_stats',
 'targets_y',
 'receiving_yards_1st_year_stats',
 'receiving_tds_1st_year_stats',
 'receiving_fumbles_1st_year_stats',
 'receiving_fumbles_lost_1st_year_stats',
 'receiving_air_yards_1st_year_stats',
 'receiving_yards_after_catch_1st_year_stats',
 'receiving_first_downs_1st_year_stats',
 'receiving_epa_1st_year_stats',
 'receiving_2pt_conversions_1st_year_stats',
 'racr_1st_year_stats',
 'target_share_1st_year_stats',
 'air_yards_share_1st_year_stats',
 'wopr_x_1st_year_stats',
 'special_teams_tds_1st_year_stats',
 'fantasy_points_1st_year_stats',
 'fantasy_points_ppr_1st_year_stats',
 'games_1st_year_stats',
 'tgt_sh_1st_year_stats',
 'ay_sh_1st_year_stats',
 'yac_sh_1st_year_stats',
 'wopr_y_1st_year_stats',
 'ry_sh_1st_year_stats',
 'rtd_sh_1st_year_stats',
 'rfd_sh_1st_year_stats',
 'rtdfd_sh_1st_year_stats',
 'dom_1st_year_stats',
 'w8dom_1st_year_stats',
 'yptmpa_1st_year_stats',
 'ppr_sh_1st_year_stats',
 'season_defensive',
 'player_game_count',
 'tackles',
 'assisted_tackles',
 'tackles_for_loss',
 'stops',
 'missed_tackle_rate',
 'missed_tackles',
 'forced_fumbles',
 'fumble_recoveries',
 'fumble_recovery_touchdowns',
 'total_defensive_pressures',
 'pass_rushing_hurries',
 'hits',
 'defensive_sacks',
 'safeties',
 'defensive_interceptions',
 'defensive_interception_touchdowns',
 'pass_break_ups',
 'coverage_targets',
 'allowed_recpetions',
 'allowed_passing_yards',
 'allowed_passing_touchdowns',
 'games_2nd_year_stats',
 'passing_yards_2nd_year_stats',
 'passing_tds_2nd_year_stats',
 'interceptions_2nd_year_stats',
 'sacks_2nd_year_stats',
 'sack_yards_2nd_year_stats',
 'passing_air_yards_2nd_year_stats',
 'passing_yards_after_catch_2nd_year_stats',
 'passing_first_downs_2nd_year_stats',
 'passing_epa_2nd_year_stats',
 'pacr_2nd_year_stats',
 'dakota_2nd_year_stats',
 'rushing_yards_2nd_year_stats',
 'rushing_tds_2nd_year_stats',
 'rushing_fumbles_2nd_year_stats',
 'rushing_fumbles_lost_2nd_year_stats',
 'rushing_first_downs_2nd_year_stats',
 'rushing_epa_2nd_year_stats',
 'targets_2nd_year_stats',
 'receptions_2nd_year_stats',
 'receiving_yards_2nd_year_stats',
 'receiving_tds_2nd_year_stats',
 'receiving_fumbles_2nd_year_stats',
 'receiving_fumbles_lost_2nd_year_stats',
 'receiving_air_yards_2nd_year_stats',
 'receiving_yards_after_catch_2nd_year_stats',
 'receiving_first_downs_2nd_year_stats',
 'receiving_epa_2nd_year_stats',
 'racr_2nd_year_stats',
 'air_yards_share_2nd_year_stats',
 'wopr_x_2nd_year_stats',
 'special_teams_tds_2nd_year_stats',
 'tgt_sh_2nd_year_stats',
 'ay_sh_2nd_year_stats',
 'yac_sh_2nd_year_stats',
 'wopr_y_2nd_year_stats',
 'ry_sh_2nd_year_stats',
 'rtd_sh_2nd_year_stats',
 'rfd_sh_2nd_year_stats',
 'dom_2nd_year_stats',
 'yptmpa_2nd_year_stats']

In [242]:
export_df = pd.DataFrame()

for pos in pos_list:
    print(pos)
    temp_df = third_year_df[third_year_df['pos'] == pos]
    temp_df = temp_df[temp_df['3rd_season'] < current_year]
    pos_df = third_year_df[(third_year_df['pos'] == pos) & (third_year_df['3rd_season'] < current_year + 1)]
    
    for stat in stats_to_perdict:
        print(stat)
        X = temp_df[x_stats]
        Y = temp_df[stat]

        gbr_model = GradientBoostingRegressor(n_estimators=(temp_df.shape[0]), learning_rate=0.1)
        gbr_model.fit(X, Y)
        gbr_predictions = gbr_model.predict(pos_df[x_stats])
        pos_df[stat + '_y_pred_gbf'] = gbr_predictions
    export_df = pd.concat([export_df,pos_df])
        
export_df

QB
games_3rd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\3853764565.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


passing_yards_3rd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\3853764565.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


passing_tds_3rd_year_stats


...

dom_3rd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\3853764565.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


yptmpa_3rd_year_stats


C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\3853764565.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df[stat + '_y_pred_gbf'] = gbr_predictions


,Unnamed: 0,index,season_1st_year_stats,cfb_id,player_name,pos,school,ht,wt,forty,...,special_teams_tds_3rd_year_stats_y_pred_gbf,tgt_sh_3rd_year_stats_y_pred_gbf,ay_sh_3rd_year_stats_y_pred_gbf,yac_sh_3rd_year_stats_y_pred_gbf,wopr_y_3rd_year_stats_y_pred_gbf,ry_sh_3rd_year_stats_y_pred_gbf,rtd_sh_3rd_year_stats_y_pred_gbf,rfd_sh_3rd_year_stats_y_pred_gbf,dom_3rd_year_stats_y_pred_gbf,yptmpa_3rd_year_stats_y_pred_gbf
12,402.0,155.0,2015.0,brett-hundley-1,Brett Hundley,QB,UCLA,75.0,226.0,4.63,...,0.0,0.002814,-1.857509e-03,0.013487,0.002728,4.851365e-03,0.000002,0.009884,0.002390,0.028142
13,432.0,191.0,2015.0,sean-mannion-1,Sean Mannion,QB,Oregon State,78.0,229.0,5.14,...,0.0,0.000001,-4.923948e-08,0.000002,0.000002,9.735133e-07,0.000002,0.000001,0.000002,0.000005
14,433.0,192.0,2015.0,marcus-mariota-1,Marcus Mariota,QB,Oregon,76.0,222.0,4.52,...,0.0,0.000001,-4.923948e-08,0.000021,0.000002,1.496727e-06,0.000002,0.000001,0.000002,0.000017
15,462.0,229.0,2015.0,bryce-petty-1,Bryce Petty,QB,Baylor,75.0,230.0,4.87,...,0.0,0.000001,-4.923948e-08,0.000007,0.000002,1.496727e-06,0.000002,0.000001,0.000002,0.000007
16,527.0,305.0,2015.0,jameis-winston-1,Jameis Winston,QB,Florida State,76.0,231.0,4.97,...,0.0,0.000001,-4.923948e-08,0.000005,0.000002,9.735133e-07,0.000002,0.000001,0.000002,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2562.0,2476.0,2022.0,teagan-quitoriano-1,Teagan Quitoriano,TE,Oregon St.,78.0,258.0,4.75,...,0.0,0.109401,9.026729e-02,0.089817,0.216377,7.917368e-02,0.125543,0.067625,0.126554,0.572156
512,2579.0,2493.0,2022.0,jeremy-ruckert-1,Jeremy Ruckert,TE,Ohio St.,77.0,250.0,4.75,...,0.0,0.040924,1.944258e-02,0.030564,0.066983,2.608297e-02,0.014557,0.047822,0.017482,0.260936
513,2613.0,2531.0,2022.0,cole-turner-2,Cole Turner,TE,Nevada,78.0,246.0,4.76,...,0.0,0.017338,4.404663e-02,0.011571,0.033028,2.364016e-02,0.014671,0.008948,0.011275,0.112026
514,2638.0,2563.0,2022.0,jelani-woods-1,Jelani Woods,TE,Virginia,79.0,259.0,4.61,...,0.0,0.085768,7.686156e-02,0.083061,0.175851,6.375082e-02,0.042141,0.065906,0.066960,0.386284


In [243]:
export_df.fillna(0)

filter_columns = ['player_name','pos','school','3rd_season']

perdicted_stats = [item + '_y_pred_gbf' for item in stats_to_perdict]

filtered_columns = filter_columns + perdicted_stats

In [244]:
filtered_export = export_df[filtered_columns]

filtered_export['scrimmage_yards'] = filtered_export['rushing_yards_3rd_year_stats_y_pred_gbf'] + filtered_export['receiving_yards_3rd_year_stats_y_pred_gbf']
filtered_export['scrimmage_TDs'] = filtered_export['rushing_tds_3rd_year_stats_y_pred_gbf'] + filtered_export['receiving_tds_3rd_year_stats_y_pred_gbf']

scoring_dict = {
'passing_yards_3rd_year_stats_y_pred_gbf': .04,
'passing_tds_3rd_year_stats_y_pred_gbf': 4,
'interceptions_3rd_year_stats_y_pred_gbf' : -2,
'rushing_yards_3rd_year_stats_y_pred_gbf' : .1,
'rushing_tds_3rd_year_stats_y_pred_gbf' : 6,
'rushing_fumbles_3rd_year_stats_y_pred_gbf' : -1,
'rushing_fumbles_lost_3rd_year_stats_y_pred_gbf' : -1,
'rushing_first_downs_3rd_year_stats_y_pred_gbf': .5,
'receptions_3rd_year_stats_y_pred_gbf' : .5,
'receiving_yards_3rd_year_stats_y_pred_gbf' : .1,
'receiving_tds_3rd_year_stats_y_pred_gbf' : 6,
'receiving_fumbles_3rd_year_stats_y_pred_gbf' : -1,
'receiving_fumbles_lost_3rd_year_stats_y_pred_gbf' : -1,
'receiving_first_downs_3rd_year_stats_y_pred_gbf' : .5
}

filtered_export['fantasy_points'] = filtered_export[scoring_dict.keys()].mul(list(scoring_dict.values()), axis=1).sum(axis=1)
filtered_export['fantasy_points/G'] = filtered_export['fantasy_points'] / filtered_export['games_3rd_year_stats_y_pred_gbf']

filtered_export = filtered_export.sort_values(by='fantasy_points/G', ascending=False)

filtered_export.to_csv('filtered_3rd_year_prediction.csv')

C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\548141496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_export['scrimmage_yards'] = filtered_export['rushing_yards_3rd_year_stats_y_pred_gbf'] + filtered_export['receiving_yards_3rd_year_stats_y_pred_gbf']
C:\Users\jkepler\AppData\Local\Temp\ipykernel_21608\548141496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_export['scrimmage_TDs'] = filtered_export['rushing_tds_3rd_year_stats_y_pred_gbf'] + filtered_export['receiving_tds_